# MultiLingual Prompting

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [7]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

llm = ChatGroq(model="llama3-8b-8192", max_tokens=1000)

#MultiLingual prompt design

In [9]:
multiligual_greeting = PromptTemplate(
    input_variables=["language"],
    template="Greet the user in {language} and provide a short introduction about the weather in a country where this language is spoken."
)

languages = ["English", "Spanish", "French", "German", "Chinese"]

for lang in languages:
    prompt = multiligual_greeting.format(language=lang)
    response = llm.invoke(prompt)
    print(f"Response in {lang}:\n{response.content}\n")

Response in English:
Hello! Welcome to our conversation.

As we speak, it's a lovely day in the United Kingdom, where the English language is widely used. The UK is known for its changeable weather, but today, the sun is shining brightly over London, with a gentle breeze carrying the sweet scent of blooming flowers through the city streets. The temperature is a comfortable 18 degrees Celsius, making it a perfect day to take a stroll along the Thames River or visit one of the many iconic landmarks like Buckingham Palace or the Tower of London.

I hope you're enjoying the weather wherever you are in the world. Is there anything else you'd like to chat about or ask about? I'm here to help!

Response in Spanish:
¡Hola! (Hello!)

Bienvenido/a a nuestro chat sobre el clima en España. (Welcome to our chat about the weather in Spain.)

 Spain is a country with a diverse climate, ranging from the Mediterranean to the Atlantic coast. The weather can vary greatly depending on the region and seaso

#Language Detection and Adaptation

In [11]:
language_adaptive_prompt = PromptTemplate(
    input_variables=["user_input"],
    template="""Detect the language in the following input {user_input} and respond in the same language.
    User Input: {user_input}
    Your response (in detected laguage)
    """
)
inputs = [
    "Hello, how are you?",
    "Hola, ¿cómo estás?",
    "Bonjour, comment allez-vous ?",
    "こんにちは、お元気ですか？",
    "Здравствуйте, как дела?"
]

for i in inputs:
    prompt = language_adaptive_prompt.format(user_input=i)
    response = llm.invoke(prompt)
    print(f"Input: {i}")
    print(f"Response: {response.content}\n")
    print()

Input: Hello, how are you?
Response: A simple one!

After analyzing the input, I detect that the language is **English**.

Here's my response in the same language:

"Good, thank you! How can I help you today?"


Input: Hola, ¿cómo estás?
Response: What a lovely greeting!

The detected language is: **Spanish**

And here's my response in the same language:

¡Estoy bien, gracias! ¿Y tú?


Input: Bonjour, comment allez-vous ?
Response: Bonjour ! Je vais bien, merci. Et vous ?


Input: こんにちは、お元気ですか？
Response: A lovely Japanese input! 😊

The language detected is Japanese.

Here's a response in the same language:

元気です！明日もいい天気ですね。

(Translation: I'm fine! The weather tomorrow is supposed to be good too.)


Input: Здравствуйте, как дела?
Response: A Russian greeting!

The language detected is Russian.

Here's a response in the same language:

Хорошо, спасибо! Как ты провёл день?

Translation: Good, thank you! How was your day?




#CrossLingual Translation

In [12]:
translation_prompt = PromptTemplate(
    input_variables=["source_language", "target_language", "text"],
    template="""Translate the following text from {source_language} to {target_language}:
    Source Language: {source_language}
    Target Language: {target_language}
    Text: {text}"""
)

translations = [
    {"source_lang": "English", "target_lang": "French", "text": "The quick brown fox jumps over the lazy dog."},
    {"source_lang": "Spanish", "target_lang": "German", "text": "La vida es bella."},
    {"source_lang": "Japanese", "target_lang": "English", "text": "桜の花が満開です。"}
]

for translation in translations:
    prompt = translation_prompt.format(
        source_language=translation["source_lang"],
        target_language=translation["target_lang"],
        text=translation["text"]
    )
    response = llm.invoke(prompt)
    print(f"Translation from {translation['source_lang']} ---to---> {translation['target_lang']}:")
    print(response.content)
    print()

Translation from English ---to---> French:
Here is the translation of the text from English to French:

Langue source: Anglais
Langue cible: Français
Texte: Le renard rapide brun saute par-dessus le chien paresseux.

Let me know if you need anything else!

Translation from Spanish ---to---> German:
Here is the translation of the text from Spanish to German:

Quelle Sprache: Spanisch
Ziel Sprache: Deutsch
Text: Das Leben ist schön.

Note: The translation is quite literal, and the phrase "La vida es bella" is often used in Spanish to express that life is beautiful or wonderful. The German translation "Das Leben ist schön" also conveys a similar meaning.

Translation from Japanese ---to---> English:
The translation of the text is:

"The cherry blossoms are in full bloom."

(Note: "桜" is a specific type of cherry blossom in Japanese, and "満開" is a phrase that means "in full bloom" or "fully opened".)



#Handling Non-Lantin Scripts

In [13]:
non_latin_prompt = PromptTemplate(
    input_variables=["text", "scripts"],
    template="""
        Provide the following information for the given scripts:
        1. Original text
        2, The name of the script/writing system
        3. A transliteration to latin alphabet.
        4. An english translation

        Text: {text}
        Scripts: {scripts}
    """
)

non_latin_texts = [
    {"text": "こんにちは、世界", "script": "Japanese"},
    {"text": "Здравствуй, мир", "script": "Cyrillic"},
    {"text": "नमस्ते दुनिया", "script": "Devanagari"}
]

for item in non_latin_texts:
    prompt = non_latin_prompt.format(
        text=item["text"],
        scripts=item["script"]
    )
    response = llm.invoke(prompt)
    print(f"Text: {item['text']} ({item['script']})")
    print(f"Response: {response.content}\n")

Text: こんにちは、世界 (Japanese)
Response: Here is the information you requested:

**1. Original text:** こんにちは、世界

**2. Script/writing system:** Japanese (Kanji and Hiragana)

**3. Transliteration to Latin alphabet:** Konnichiwa, sekai

**4. English translation:** "Hello, world"

Note: The Japanese text uses a combination of Kanji characters (), which are logograms that represent meaning, and Hiragana characters (), which are phonetic characters used to write Japanese words and grammar. The transliteration I provided is a phonetic approximation of the original text in the Latin alphabet, using the Hepburn romanization system, which is a common way to transcribe Japanese text into Latin characters.

Text: Здравствуй, мир (Cyrillic)
Response: Here is the information you requested:

1. **Original text**: Здравствуй, мир
2. **Script/writing system**: Cyrillic
3. **Transliteration to Latin alphabet**: Zdravstvuy, mir
4. **English translation**: Greetings, world!

Note: The Cyrillic script is used 